In [1]:
from transformers import pipeline 

In [2]:
model_name = "r1char9/rubert-base-cased-russian-sentiment"

classifier = pipeline("sentiment-analysis", model=model_name)

text = "Video is okay! I think its pretty good!"

result = classifier(text)

print(result)

[{'label': 'positive', 'score': 0.9461501836776733}]


In [3]:
from transformers import pipeline

language_detector = pipeline('text-classification', model='papluca/xlm-roberta-base-language-detection')

rus_toxicity_classifier = pipeline('text-classification', model='cointegrated/rubert-tiny-toxicity')

rus_sentiment_classifier = pipeline('sentiment-analysis', model='r1char9/rubert-base-cased-russian-sentiment')

# multi_toxicity_classifier = pipeline('text-classification', model='s-nlp/xlmr_formality_toxic_multilingual')

multi_sentiment_classifier = pipeline('sentiment-analysis', model='tabularisai/multilingual-sentiment-analysis')

print("All the models succesfully loaded!")



def analyze_comment(comment_text: str) -> dict:
    
    lang_result = language_detector(comment_text, top_k=1)
    language = lang_result[0]['label']
    
    toxicity = "N/A"  
    sentiment = "N/A"

    if language == 'ru':
        toxicity_result = rus_toxicity_classifier(comment_text)
        toxicity = toxicity_result[0]['label']

        sentiment_result = rus_sentiment_classifier(comment_text)
        sentiment = sentiment_result[0]['label']

    elif language == 'kk': 
        # toxicity_result = multi_toxicity_classifier(comment_text)
        # toxicity = toxicity_result[0]['label']
        
        sentiment_result = multi_sentiment_classifier(comment_text)
        sentiment = sentiment_result[0]['label']
    
    final_analysis = {
        'text': comment_text,
        'language': language,
        'toxicity': toxicity,
        'sentiment': sentiment
    }
    
    return final_analysis

if __name__ == '__main__':
    test_comment_ru_ok = "Отличный сервис, скорость интернета просто летает! Спасибо!"
    test_comment_ru_toxic = "Что за отвратительная связь, вы надоели уже со своими обещаниями!"
    test_comment_kz = "Керемет! Маған бәрі ұнады, рахмет сіздерге."
    test_comment_en = "This is a great service, thank you!"
    
    print("\n--- Testing: ... ---")
    
    analysis1 = analyze_comment(test_comment_ru_ok)
    print(f"\nАнализ комментария 1: {analysis1}")

    analysis2 = analyze_comment(test_comment_ru_toxic)
    print(f"Анализ комментария 2: {analysis2}")
    
    analysis3 = analyze_comment(test_comment_kz)
    print(f"Анализ комментария 3: {analysis3}")

    analysis4 = analyze_comment(test_comment_en)
    print(f"Анализ комментария 4: {analysis4}")

Загрузка моделей... Это может занять несколько минут.
Все рабочие модели успешно загружены!

--- Тестирование аналитического конвейера ---

Анализ комментария 1: {'text': 'Отличный сервис, скорость интернета просто летает! Спасибо!', 'language': 'ru', 'toxicity': 'non-toxic', 'sentiment': 'positive'}
Анализ комментария 2: {'text': 'Что за отвратительная связь, вы надоели уже со своими обещаниями!', 'language': 'ru', 'toxicity': 'non-toxic', 'sentiment': 'negative'}
Анализ комментария 3: {'text': 'Керемет! Маған бәрі ұнады, рахмет сіздерге.', 'language': 'tr', 'toxicity': 'N/A', 'sentiment': 'N/A'}
Анализ комментария 4: {'text': 'This is a great service, thank you!', 'language': 'en', 'toxicity': 'N/A', 'sentiment': 'N/A'}


In [4]:
pip install langdetect

Note: you may need to restart the kernel to use updated packages.


In [51]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

print("Загрузка моделей... Это может занять несколько минут.")

language_detector = pipeline('text-classification', model='papluca/xlm-roberta-base-language-detection')

translator_tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
translator_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

rus_toxicity_classifier = pipeline('text-classification', model='cointegrated/rubert-tiny-toxicity')
rus_sentiment_classifier = pipeline('sentiment-analysis', model='r1char9/rubert-base-cased-russian-sentiment')

en_toxicity_classifier = pipeline('text-classification', model='martin-ha/toxic-comment-model')
en_sentiment_classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

zero_shot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

print("All models succesfully loaded!")

SPAM_MARKERS = {"подпис", "канал", "заход", "переход", "профил", "ссылк", "заработ", "крипт"}
URL_PATTERN = re.compile(r'https?://\S+|www\.\S+')


def clean_text(text: str) -> str:
    text = re.sub(r'[^\w\s]', '', text)
    return " ".join(text.split())

def is_kazakh(text: str) -> bool:
    KAZAKH_CHARS = set("ӘәҒғҚқҢңӨөҰұҮүҺһІі")
    return any(char in KAZAKH_CHARS for char in text)

def translate_to_russian(text: str, src_lang_code: str = "kaz_Cyrl") -> str:
    try:
        translator_tokenizer.src_lang = src_lang_code
        encoded_text = translator_tokenizer(text, return_tensors="pt")
        target_lang_id = translator_tokenizer.get_lang_id("rus_Cyrl")
        generated_tokens = translator_model.generate(**encoded_text, forced_bos_token_id=target_lang_id)
        return translator_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    except Exception as e:
        print(f"Translation error: {e}")
        return text

def detect_spam_by_rules(text: str) -> bool:
    text_lower = text.lower()
    if URL_PATTERN.search(text_lower):
        return True
    if any(marker in text_lower for marker in SPAM_MARKERS):
        return True
    return False

def get_moderation_verdict(text: str, language: str) -> str:
    if detect_spam_by_rules(text):
        return 'spam'
    
    toxicity_result = 'non-toxic'
    if language in ['ru', 'kk']:
        toxicity_result = rus_toxicity_classifier(text)[0]['label']
    elif language == 'en':
        toxicity_result = en_toxicity_classifier(text)[0]['label']
    
    return 'insult' if toxicity_result == 'toxic' else 'ok'



def analyze_comment(comment_text: str) -> dict:
    
    cleaned_text = clean_text(comment_text)
    if not cleaned_text:
        return {
            'text': comment_text, 'language': 'unknown', 'moderation_verdict': 'ok',
            'sentiment': 'N/A', 'comment_type': 'N/A'
        }

    if is_kazakh(cleaned_text):
        detected_language = 'kk'
    else:
        lang_results = language_detector(cleaned_text, top_k=1)
        detected_language = lang_results[0]['label']
            
    sentiment = "N/A"
    comment_type = "N/A"
    text_to_analyze = cleaned_text
    
    if detected_language == 'kk':
        text_to_analyze = translate_to_russian(cleaned_text, src_lang_code="kaz_Cyrl")

    moderation_verdict = get_moderation_verdict(text_to_analyze, detected_language)

    if moderation_verdict != 'spam':
        if detected_language in ['ru', 'kk']:
            sentiment = rus_sentiment_classifier(text_to_analyze)[0]['label']
        elif detected_language == 'en':
            sentiment = en_sentiment_classifier(text_to_analyze)[0]['label']
        
        descriptive_labels = [
         "user asks a question for a telecommunications company", "user complains about a telecommunications company's service",
        "user expresses gratitude to a telecommunications company", "user shares their opinion about a telecommunications company's service"
        ]
        label_map = {
            "user asks a question for a telecommunications company": "question",
            "user complains about a telecommunications company's service": "complaint",
            "user expresses gratitude to a telecommunications company": "gratitude",
            "user shares their opinion about a telecommunications company's service": "feedback"
        }
        
        type_result = zero_shot_classifier(text_to_analyze, descriptive_labels)
        top_label = type_result['labels'][0]
        comment_type = label_map.get(top_label, "feedback")

    final_analysis = {
        'text': comment_text,
        'language': detected_language,
        'moderation_verdict': moderation_verdict,
        'sentiment': sentiment,
        'comment_type': comment_type
    }
    return final_analysis

if __name__ == '__main__':
    test_q = "Подскажите, как сменить тариф?"
    test_g = "Спасибо вам большое, все отлично работает!"
    test_c = "Ужасный интернет, постоянно пропадает!"
    test_f = "Пользуюсь уже год, в целом нормально."
    test_spam = "Заходи на мой канал про крипту www.example.com"
    test_kz = "Керемет! Маған бәрі ұнады, рахмет сіздерге."
    test_rus = "Почему интернет так тупит?"
    
    print("\n--- Тестирование финального аналитического конвейера ---")
    print(f"\nВопрос: {analyze_comment(test_q)}")
    print(f"Благодарность: {analyze_comment(test_g)}")
    print(f"Жалоба: {analyze_comment(test_c)}")
    print(f"Отзыв: {analyze_comment(test_f)}")
    print(f"Спам: {analyze_comment(test_spam)}")
    print(f"Казахский: {analyze_comment(test_kz)}")
    print(f"Русский: {analyze_comment(test_rus)}")

Загрузка моделей... Это может занять несколько минут.
Все рабочие модели успешно загружены!

--- Тестирование финального аналитического конвейера ---

Вопрос: {'text': 'Подскажите, как сменить тариф?', 'language': 'ru', 'moderation_verdict': 'ok', 'sentiment': 'neutral', 'comment_type': 'feedback'}
Благодарность: {'text': 'Спасибо вам большое, все отлично работает!', 'language': 'ru', 'moderation_verdict': 'ok', 'sentiment': 'positive', 'comment_type': 'gratitude'}
Жалоба: {'text': 'Ужасный интернет, постоянно пропадает!', 'language': 'ru', 'moderation_verdict': 'ok', 'sentiment': 'negative', 'comment_type': 'feedback'}
Отзыв: {'text': 'Пользуюсь уже год, в целом нормально.', 'language': 'ru', 'moderation_verdict': 'ok', 'sentiment': 'positive', 'comment_type': 'feedback'}
Спам: {'text': 'Заходи на мой канал про крипту www.example.com', 'language': 'ru', 'moderation_verdict': 'spam', 'sentiment': 'N/A', 'comment_type': 'N/A'}
Ошибка перевода: 'NllbTokenizerFast' object has no attribute